In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
#import matplotlib as plt
import matplotlib.pyplot as plt
import sklearn.preprocessing as sp
%matplotlib inline
sns.set_style('whitegrid')
sns.set_context("poster")
sns.set_context("talk", 1.0, {"lines.linewidth": 3})

In [2]:
#パターン18のデータを取得
df18_1 = pd.read_csv("./取得データセット/受信機1/nkmr1-dataset18.csv")
df18_2 = pd.read_csv("./取得データセット/受信機2/nkmr2-dataset18.csv")
df18_3 = pd.read_csv("./取得データセット/受信機3/nkmr3-dataset18.csv")
df18_4 = pd.read_csv("./取得データセット/受信機4/nkmr4-dataset18.csv")

#それぞれのカラムを作成
df18_1.columns = ["UUIDs","Major","Minor","Accuracy","Proximity","TxPower","RSSI","device_name","timestamp","x","y"]
df18_2.columns = ["UUIDs","Major","Minor","Accuracy","Proximity","TxPower","RSSI","device_name","timestamp","x","y"]
df18_3.columns = ["UUIDs","Major","Minor","Accuracy","Proximity","TxPower","RSSI","device_name","timestamp","x","y"]
df18_4.columns = ["UUIDs","Major","Minor","Accuracy","Proximity","TxPower","RSSI","device_name","timestamp","x","y"]

#それぞれの受信機のデータを統合
df18 = df18_1.append(df18_2).append(df18_3).append(df18_4)

#deviceを作成
df18["device"] = 0
df18.device = df18.device.where(df18.device_name != "rasp1",1)
df18.device = df18.device.where(df18.device_name != "nkmr-2",2)
df18.device = df18.device.where(df18.device_name != "rasp3",3)
df18.device = df18.device.where(df18.device_name != "nkmr-4",4)

#同期ビーコンのid
set_uuid = "1000000007ae1001b000001c4d8dffff"

#送信機のid
id1 = "10000000aaaa41441111110000000000"
id2 = "20000000aaaa41441111110000000000"
id3 = "30000000aaaa41441111110000000000"
id4 = "40000000aaaa41441111110000000000"
id5 = "50000000aaaa41441111110000000000"
id6 = "60000000aaaa41441111110000000000"
id7 = "70000000aaaa41441111110000000000"
id8 = "80000000aaaa41441111110000000000" 
id9 = "90000000aaaa41441111110000000000"
id10 = "a0000000aaaa41441111110000000000"

#今回はid5は抜く
#locateを作成
df18["locate"] = 0
df18.locate = df18.locate.where(df18.UUIDs != id1,1)
df18.locate = df18.locate.where(df18.UUIDs != id2,2)
df18.locate = df18.locate.where(df18.UUIDs != id3,3)
df18.locate = df18.locate.where(df18.UUIDs != id4,4)
df18.locate = df18.locate.where(df18.UUIDs != id5,5)
df18.locate = df18.locate.where(df18.UUIDs != id6,6)
df18.locate = df18.locate.where(df18.UUIDs != id7,7)
df18.locate = df18.locate.where(df18.UUIDs != id8,8)
df18.locate = df18.locate.where(df18.UUIDs != id9,9)
df18.locate = df18.locate.where(df18.UUIDs != id10,10)
df18.locate = df18.locate.astype(int)
#df18 = df18[df18.UUIDs != id5]

#発信機ごとに分ける
df18_1 = df18[(df18.UUIDs == id1) | (df18.UUIDs == set_uuid)]
df18_2 = df18[(df18.UUIDs == id2) | (df18.UUIDs == set_uuid)]
df18_3 = df18[(df18.UUIDs == id3) | (df18.UUIDs == set_uuid)]
df18_4 = df18[(df18.UUIDs == id4) | (df18.UUIDs == set_uuid)]
df18_5 = df18[(df18.UUIDs == id5) | (df18.UUIDs == set_uuid)]
df18_6 = df18[(df18.UUIDs == id6) | (df18.UUIDs == set_uuid)]
df18_7 = df18[(df18.UUIDs == id7) | (df18.UUIDs == set_uuid)]
df18_8 = df18[(df18.UUIDs == id8) | (df18.UUIDs == set_uuid)]
df18_9 = df18[(df18.UUIDs == id9) | (df18.UUIDs == set_uuid)]
df18_10 = df18[(df18.UUIDs == id10) | (df18.UUIDs == set_uuid)]

df_list = [df18_1,df18_2,df18_3,df18_4,df18_5,df18_6,df18_7,df18_8,df18_9,df18_10]

In [3]:
df18[df18.UUIDs != set_uuid].count()

UUIDs          266583
Major          266583
Minor          266583
Accuracy       266583
Proximity      266583
TxPower        266583
RSSI           266583
device_name    266583
timestamp      266583
x              266583
y              266583
device         266583
locate         266583
dtype: int64

### 時刻合わせ

In [4]:
#　オブジェクト型の目的変数を数値情報に変換
import sklearn.preprocessing as sp

# 前処理用のインスタンスを宣言
le = sp.LabelEncoder()

def set_time(data,set_uuid):
    """
    時刻合わせを行う
    第一引数はdataframe
    第２引数は時刻合わせを行うUUID
    """
    # 時刻合わせ用のビーコンを1秒間隔に設定
    set_time_df = data[data.UUIDs == set_uuid].drop_duplicates(['Minor'],keep = "first")
    # 時刻合わせ用ビーコン以外を定義
    df = data[data.UUIDs != set_uuid]
    # データを結合してインデックスでソート
    df = df.append(set_time_df).sort_index(ascending=True)
    # 時刻合わせ用ビーコンを取得した行にtimeカラムを設定
    df["time"] = df["Minor"].where(df["UUIDs"] == set_uuid)
    # 時刻合わせを行い欠損値がある部分は削除
    df = df.fillna(method="ffill").dropna()
    # int型に変換
    df["time"] = df["time"].astype(int)
    # 時刻合わせ用のビーコンデータ以外
    df = df[df["UUIDs"] != set_uuid]
    
    return df

# 前処理を各データフレームに行う
for (i,df) in enumerate(df_list):
    df_list[i] = set_time(df,set_uuid)
    # device_nameを変換
    #df_list[i]["device"] = le.fit_transform(df_list[i][df_list[i].columns[7]]) + 1

In [5]:
df_list[2] = df_list[2][df_list[2].device != 1]

### 直接波と反射波を識別する

In [6]:
# KMeansライブラリをインポート
from sklearn.cluster import KMeans
from IPython.display import display

In [7]:
def make_cluster(df_list,colum,*,N_CLUSTERS=2):
    """
          反射波と入射波を判別するクラスタを作成し描写を行う
          第1引数(地点毎のデータフレームが入ったリスト)
          第２引数(クラスタリングを行いたいカラムのリスト)
          第３引数(クラスタ数)
          データフレーム型でクラスタリング結果を返す
    """

    # 空のデータフレームを用意
    df_cluster = pd.DataFrame()
    concat_list = []
    #座標ごとに
    for df in df_list: 
        #受信機ごとに
        for dev in df.device_name.unique():
            #受信機ごとに分けたdataframe
            df_dev = df[df.device_name == dev].copy()
            display(df_dev)
            # 指定されたカラムを利用してクラスタリング
            pred = KMeans(n_clusters=N_CLUSTERS).fit_predict(np.array(df_dev[colum]))
            # clusterカラムを作成し予測結果を代入
            df_dev.loc[:,("cluster")] = pred
            # RSSI値の平均値を比較して入射波と反射波の判定を行う
            if df_dev[df_dev["cluster"] == 0]["RSSI"].mean() > df_dev[df_dev["cluster"] == 1]["RSSI"].mean():
                pass
            else:
                df_dev.loc[df_dev["cluster"] == 0,"cluster"] = 3
                df_dev.loc[df_dev["cluster"] == 1,"cluster"] = 0
                df_dev.loc[df_dev["cluster"] == 3,"cluster"] = 1
            # 予測結果を格納
            df_cluster = pd.concat([df_cluster,df_dev],ignore_index=True)
        concat_list.append(df_cluster)
        # 空のデータフレームを用意
        df_cluster = pd.DataFrame()

    return concat_list

In [8]:
df_list = make_cluster(df_list,["RSSI","locate"]) 

,UUIDs,Major,Minor,Accuracy,Proximity,TxPower,RSSI,device_name,timestamp,x,y,device,locate,time
36,10000000aaaa41441111110000000000,2,38,0.715691,near,-78,-71,nkmr-2,2018/02/01 17:40:21,200,200,2,1,4
45,10000000aaaa41441111110000000000,2,38,1.154145,near,-78,-81,nkmr-2,2018/02/01 17:40:21,200,200,2,1,4
55,10000000aaaa41441111110000000000,2,38,0.908852,near,-78,-76,nkmr-2,2018/02/01 17:40:21,200,200,2,1,4
59,10000000aaaa41441111110000000000,2,38,1.210637,near,-78,-82,nkmr-2,2018/02/01 17:40:21,200,200,2,1,4
63,10000000aaaa41441111110000000000,2,38,0.908852,near,-78,-76,nkmr-2,2018/02/01 17:40:22,200,200,2,1,5
70,10000000aaaa41441111110000000000,2,38,1.154145,near,-78,-81,nkmr-2,2018/02/01 17:40:22,200,200,2,1,5
75,10000000aaaa41441111110000000000,2,38,0.953337,near,-78,-77,nkmr-2,2018/02/01 17:40:22,200,200,2,1,5
80,10000000aaaa41441111110000000000,2,38,0.715691,near,-78,-71,nkmr-2,2018/02/01 17:40:22,200,200,2,1,5
84,10000000aaaa41441111110000000000,2,38,2.363539,near,-78,-96,nkmr-2,2018/02/01 17:40:22,200,200,2,1,5
90,10000000aaaa41441111110000000000,2,38,1.210637,near,-78,-82,nkmr-2,2018/02/01 17:40:22,200,200,2,1,5


,UUIDs,Major,Minor,Accuracy,Proximity,TxPower,RSSI,device_name,timestamp,x,y,device,locate,time
37,10000000aaaa41441111110000000000,2,38,1.465642,near,-78,-86,rasp3,2018/02/07 16:37:32,200,200,3,1,4
46,10000000aaaa41441111110000000000,2,38,0.750722,near,-78,-72,rasp3,2018/02/07 16:37:33,200,200,3,1,4
69,10000000aaaa41441111110000000000,2,38,0.908852,near,-78,-76,rasp3,2018/02/07 16:37:33,200,200,3,1,5
83,10000000aaaa41441111110000000000,2,38,1.537381,near,-78,-87,rasp3,2018/02/07 16:37:33,200,200,3,1,5
111,10000000aaaa41441111110000000000,2,38,1.465642,near,-78,-86,rasp3,2018/02/07 16:37:34,200,200,3,1,6
120,10000000aaaa41441111110000000000,2,38,1.537381,near,-78,-87,rasp3,2018/02/07 16:37:34,200,200,3,1,6
153,10000000aaaa41441111110000000000,2,38,1.465642,near,-78,-86,rasp3,2018/02/07 16:37:35,200,200,3,1,8
166,10000000aaaa41441111110000000000,2,38,1.537381,near,-78,-87,rasp3,2018/02/07 16:37:35,200,200,3,1,8
178,10000000aaaa41441111110000000000,2,38,0.908852,near,-78,-76,rasp3,2018/02/07 16:37:36,200,200,3,1,8
186,10000000aaaa41441111110000000000,2,38,1.332051,near,-78,-84,rasp3,2018/02/07 16:37:36,200,200,3,1,8


,UUIDs,Major,Minor,Accuracy,Proximity,TxPower,RSSI,device_name,timestamp,x,y,device,locate,time
38,10000000aaaa41441111110000000000,2,38,2.363539,near,-78,-96,nkmr-4,2018/02/09 09:36:39,200,200,4,1,4
81,10000000aaaa41441111110000000000,2,38,1.154145,near,-78,-81,nkmr-4,2018/02/09 09:36:39,200,200,4,1,5
88,10000000aaaa41441111110000000000,2,38,2.148106,near,-78,-94,nkmr-4,2018/02/09 09:36:39,200,200,4,1,5
99,10000000aaaa41441111110000000000,2,38,2.148106,near,-78,-94,nkmr-4,2018/02/09 09:36:40,200,200,4,1,6
123,10000000aaaa41441111110000000000,2,38,2.047870,near,-78,-93,nkmr-4,2018/02/09 09:36:40,200,200,4,1,7
142,10000000aaaa41441111110000000000,2,38,1.952310,near,-78,-92,nkmr-4,2018/02/09 09:36:40,200,200,4,1,7
146,10000000aaaa41441111110000000000,2,38,0.787467,near,-78,-73,nkmr-4,2018/02/09 09:36:40,200,200,4,1,7
155,10000000aaaa41441111110000000000,2,38,1.269894,near,-78,-83,nkmr-4,2018/02/09 09:36:41,200,200,4,1,8
161,10000000aaaa41441111110000000000,2,38,0.908852,near,-78,-76,nkmr-4,2018/02/09 09:36:41,200,200,4,1,8
167,10000000aaaa41441111110000000000,2,38,1.332051,near,-78,-84,nkmr-4,2018/02/09 09:36:41,200,200,4,1,8


,UUIDs,Major,Minor,Accuracy,Proximity,TxPower,RSSI,device_name,timestamp,x,y,device,locate,time
38,10000000aaaa41441111110000000000,2,38,1.861210,near,-78,-91,rasp1,2018/02/06 16:38:09,200,200,1,1,4
101,10000000aaaa41441111110000000000,2,38,2.047870,near,-78,-93,rasp1,2018/02/06 16:38:12,200,200,1,1,6
114,10000000aaaa41441111110000000000,2,38,2.047870,near,-78,-93,rasp1,2018/02/06 16:38:12,200,200,1,1,6
253,10000000aaaa41441111110000000000,2,38,1.952310,near,-78,-92,rasp1,2018/02/06 16:38:17,200,200,1,1,11
287,10000000aaaa41441111110000000000,2,38,2.047870,near,-78,-93,rasp1,2018/02/06 16:38:17,200,200,1,1,12
346,10000000aaaa41441111110000000000,2,38,1.774360,near,-78,-90,rasp1,2018/02/06 16:38:19,200,200,1,1,14
460,10000000aaaa41441111110000000000,2,38,2.047870,near,-78,-93,rasp1,2018/02/06 16:38:24,200,200,1,1,18
478,10000000aaaa41441111110000000000,2,38,1.952310,near,-78,-92,rasp1,2018/02/06 16:38:24,200,200,1,1,18
482,10000000aaaa41441111110000000000,2,38,1.952310,near,-78,-92,rasp1,2018/02/06 16:38:24,200,200,1,1,19
538,10000000aaaa41441111110000000000,2,38,1.774360,near,-78,-90,rasp1,2018/02/06 16:38:26,200,200,1,1,20


,UUIDs,Major,Minor,Accuracy,Proximity,TxPower,RSSI,device_name,timestamp,x,y,device,locate,time
22,20000000aaaa41441111110000000000,2,40,2.047870,near,-78,-93,nkmr-2,2018/02/01 17:40:21,200,200,2,2,3
26,20000000aaaa41441111110000000000,2,40,0.787467,near,-78,-73,nkmr-2,2018/02/01 17:40:21,200,200,2,2,3
40,20000000aaaa41441111110000000000,2,40,1.465642,near,-78,-86,nkmr-2,2018/02/01 17:40:21,200,200,2,2,4
67,20000000aaaa41441111110000000000,2,40,0.750722,near,-78,-72,nkmr-2,2018/02/01 17:40:22,200,200,2,2,5
72,20000000aaaa41441111110000000000,2,40,0.750722,near,-78,-72,nkmr-2,2018/02/01 17:40:22,200,200,2,2,5
86,20000000aaaa41441111110000000000,2,40,2.148106,near,-78,-94,nkmr-2,2018/02/01 17:40:22,200,200,2,2,5
91,20000000aaaa41441111110000000000,2,40,1.691564,near,-78,-89,nkmr-2,2018/02/01 17:40:22,200,200,2,2,6
99,20000000aaaa41441111110000000000,2,40,0.750722,near,-78,-72,nkmr-2,2018/02/01 17:40:22,200,200,2,2,6
109,20000000aaaa41441111110000000000,2,40,1.332051,near,-78,-84,nkmr-2,2018/02/01 17:40:23,200,200,2,2,6
142,20000000aaaa41441111110000000000,2,40,0.787467,near,-78,-73,nkmr-2,2018/02/01 17:40:23,200,200,2,2,7


...

,UUIDs,Major,Minor,Accuracy,Proximity,TxPower,RSSI,device_name,timestamp,x,y,device,locate,time
29,90000000aaaa41441111110000000000,2,33,1.000000,near,-78,-78,rasp3,2018/02/07 16:37:32,200,200,3,9,3
33,90000000aaaa41441111110000000000,2,33,0.908852,near,-78,-76,rasp3,2018/02/07 16:37:32,200,200,3,9,3
38,90000000aaaa41441111110000000000,2,33,1.691564,near,-78,-89,rasp3,2018/02/07 16:37:33,200,200,3,9,4
54,90000000aaaa41441111110000000000,2,33,2.047870,near,-78,-93,rasp3,2018/02/07 16:37:33,200,200,3,9,4
61,90000000aaaa41441111110000000000,2,33,1.100289,near,-78,-80,rasp3,2018/02/07 16:37:33,200,200,3,9,5
70,90000000aaaa41441111110000000000,2,33,0.866442,near,-78,-75,rasp3,2018/02/07 16:37:33,200,200,3,9,5
76,90000000aaaa41441111110000000000,2,33,1.269894,near,-78,-83,rasp3,2018/02/07 16:37:33,200,200,3,9,5
93,90000000aaaa41441111110000000000,2,33,1.210637,near,-78,-82,rasp3,2018/02/07 16:37:34,200,200,3,9,6
119,90000000aaaa41441111110000000000,2,33,2.047870,near,-78,-93,rasp3,2018/02/07 16:37:34,200,200,3,9,6
130,90000000aaaa41441111110000000000,2,33,1.774360,near,-78,-90,rasp3,2018/02/07 16:37:35,200,200,3,9,7


,UUIDs,Major,Minor,Accuracy,Proximity,TxPower,RSSI,device_name,timestamp,x,y,device,locate,time
22,a0000000aaaa41441111110000000000,2,31,1.154145,near,-78,-81,rasp1,2018/02/06 16:38:09,200,200,1,10,3
24,a0000000aaaa41441111110000000000,2,31,1.048947,near,-78,-79,rasp1,2018/02/06 16:38:09,200,200,1,10,3
32,a0000000aaaa41441111110000000000,2,31,1.397251,near,-78,-85,rasp1,2018/02/06 16:38:09,200,200,1,10,3
39,a0000000aaaa41441111110000000000,2,31,1.612630,near,-78,-88,rasp1,2018/02/06 16:38:09,200,200,1,10,4
43,a0000000aaaa41441111110000000000,2,31,1.154145,near,-78,-81,rasp1,2018/02/06 16:38:09,200,200,1,10,4
47,a0000000aaaa41441111110000000000,2,31,1.465642,near,-78,-86,rasp1,2018/02/06 16:38:09,200,200,1,10,4
50,a0000000aaaa41441111110000000000,2,31,1.210637,near,-78,-82,rasp1,2018/02/06 16:38:10,200,200,1,10,4
51,a0000000aaaa41441111110000000000,2,31,1.612630,near,-78,-88,rasp1,2018/02/06 16:38:10,200,200,1,10,4
57,a0000000aaaa41441111110000000000,2,31,1.210637,near,-78,-82,rasp1,2018/02/06 16:38:10,200,200,1,10,4
63,a0000000aaaa41441111110000000000,2,31,1.154145,near,-78,-81,rasp1,2018/02/06 16:38:10,200,200,1,10,5


,UUIDs,Major,Minor,Accuracy,Proximity,TxPower,RSSI,device_name,timestamp,x,y,device,locate,time
24,a0000000aaaa41441111110000000000,2,31,2.363539,near,-78,-96,nkmr-2,2018/02/01 17:40:21,200,200,2,10,3
32,a0000000aaaa41441111110000000000,2,31,1.952310,near,-78,-92,nkmr-2,2018/02/01 17:40:21,200,200,2,10,3
35,a0000000aaaa41441111110000000000,2,31,1.332051,near,-78,-84,nkmr-2,2018/02/01 17:40:21,200,200,2,10,4
54,a0000000aaaa41441111110000000000,2,31,1.861210,near,-78,-91,nkmr-2,2018/02/01 17:40:21,200,200,2,10,4
61,a0000000aaaa41441111110000000000,2,31,1.861210,near,-78,-91,nkmr-2,2018/02/01 17:40:22,200,200,2,10,5
69,a0000000aaaa41441111110000000000,2,31,0.908852,near,-78,-76,nkmr-2,2018/02/01 17:40:22,200,200,2,10,5
79,a0000000aaaa41441111110000000000,2,31,1.210637,near,-78,-82,nkmr-2,2018/02/01 17:40:22,200,200,2,10,5
89,a0000000aaaa41441111110000000000,2,31,1.269894,near,-78,-83,nkmr-2,2018/02/01 17:40:22,200,200,2,10,5
105,a0000000aaaa41441111110000000000,2,31,1.861210,near,-78,-91,nkmr-2,2018/02/01 17:40:23,200,200,2,10,6
130,a0000000aaaa41441111110000000000,2,31,1.269894,near,-78,-83,nkmr-2,2018/02/01 17:40:23,200,200,2,10,7


,UUIDs,Major,Minor,Accuracy,Proximity,TxPower,RSSI,device_name,timestamp,x,y,device,locate,time
25,a0000000aaaa41441111110000000000,2,31,0.953337,near,-78,-77,nkmr-4,2018/02/09 09:36:38,200,200,4,10,3
45,a0000000aaaa41441111110000000000,2,31,1.100289,near,-78,-80,nkmr-4,2018/02/09 09:36:39,200,200,4,10,4
54,a0000000aaaa41441111110000000000,2,31,1.269894,near,-78,-83,nkmr-4,2018/02/09 09:36:39,200,200,4,10,4
66,a0000000aaaa41441111110000000000,2,31,1.210637,near,-78,-82,nkmr-4,2018/02/09 09:36:39,200,200,4,10,5
82,a0000000aaaa41441111110000000000,2,31,1.100289,near,-78,-80,nkmr-4,2018/02/09 09:36:39,200,200,4,10,5
100,a0000000aaaa41441111110000000000,2,31,2.253249,near,-78,-95,nkmr-4,2018/02/09 09:36:40,200,200,4,10,6
108,a0000000aaaa41441111110000000000,2,31,0.826011,near,-78,-74,nkmr-4,2018/02/09 09:36:40,200,200,4,10,6
113,a0000000aaaa41441111110000000000,2,31,1.691564,near,-78,-89,nkmr-4,2018/02/09 09:36:40,200,200,4,10,6
124,a0000000aaaa41441111110000000000,2,31,1.332051,near,-78,-84,nkmr-4,2018/02/09 09:36:40,200,200,4,10,7
137,a0000000aaaa41441111110000000000,2,31,1.952310,near,-78,-92,nkmr-4,2018/02/09 09:36:40,200,200,4,10,7


,UUIDs,Major,Minor,Accuracy,Proximity,TxPower,RSSI,device_name,timestamp,x,y,device,locate,time
25,a0000000aaaa41441111110000000000,2,31,1.210637,near,-78,-82,rasp3,2018/02/07 16:37:32,200,200,3,10,3
36,a0000000aaaa41441111110000000000,2,31,1.154145,near,-78,-81,rasp3,2018/02/07 16:37:32,200,200,3,10,4
60,a0000000aaaa41441111110000000000,2,31,2.363539,near,-78,-96,rasp3,2018/02/07 16:37:33,200,200,3,10,5
68,a0000000aaaa41441111110000000000,2,31,1.397251,near,-78,-85,rasp3,2018/02/07 16:37:33,200,200,3,10,5
75,a0000000aaaa41441111110000000000,2,31,1.210637,near,-78,-82,rasp3,2018/02/07 16:37:33,200,200,3,10,5
82,a0000000aaaa41441111110000000000,2,31,1.332051,near,-78,-84,rasp3,2018/02/07 16:37:33,200,200,3,10,5
92,a0000000aaaa41441111110000000000,2,31,1.774360,near,-78,-90,rasp3,2018/02/07 16:37:34,200,200,3,10,6
97,a0000000aaaa41441111110000000000,2,31,2.047870,near,-78,-93,rasp3,2018/02/07 16:37:34,200,200,3,10,6
107,a0000000aaaa41441111110000000000,2,31,1.612630,near,-78,-88,rasp3,2018/02/07 16:37:34,200,200,3,10,6
148,a0000000aaaa41441111110000000000,2,31,1.465642,near,-78,-86,rasp3,2018/02/07 16:37:35,200,200,3,10,7


In [9]:
df_list[2][df_list[2].device == 1]

,UUIDs,Major,Minor,Accuracy,Proximity,TxPower,RSSI,device_name,timestamp,x,y,device,locate,time,cluster


### 1秒ごとのRSSI値の平均を取る

In [10]:
def max_value(df_list):
    # timeのmax値を計算
    for df in df_list:
        max_value = 0
        if max_value < df["time"].max():
            max_value = df["time"].max()
    return max_value

#座標として、引数にx,yを追加
def get_mean(df,colum_name,locate,x,y,time_max):    
    """
    処理時間かかります。
    時刻毎の中央値をとる
    第一引数、データフレーム
    第２引数、作成したいカラム名
    """
    # 空のデータフレームとリストを用意
    Direct_list = []
    Reflect_list = []
    mid_df = pd.DataFrame()
    
    # timeのユニーク数だけ
    for time_num in range(time_max): 
        # 受信機の数だけ
        for i in range(len(df_list[0]["device"].unique())): 
            # timeラベルが同じ値に対して平均値をとる
            Direct = df[(df["time"] == time_num) & (df["device"] == (i+1)) & (df["cluster"] == 0)]["RSSI"].mean()
            Reflect = df[(df["time"] == time_num) & (df["device"] == (i+1)) & (df["cluster"] == 1)]["RSSI"].mean()
            # 平均値をリストに追加する
            Direct_list.append(Direct)
            Reflect_list.append(Reflect)
        # 平均値を追加したリストに(時間、地点、クラスタ番号を追加)
        Direct_list.extend([time_num,locate,x,y,0])
        Reflect_list.extend([time_num,locate,x,y,1])
        # データフレームに各要素を追加
        mid_df = mid_df.append(pd.Series(Direct_list),ignore_index=True)
        mid_df = mid_df.append(pd.Series(Reflect_list),ignore_index=True)
        # 平均値のリストを初期化
        Direct_list = []
        Reflect_list = []
    # カラム名を定義
    mid_df.columns = [colum_name]
    
    return mid_df

#前処理を各データフレームと地点毎に行う
train_df = pd.DataFrame()
column_name = ["dev1","dev2","dev3","dev4","time","locate","x","y","cluster"]
#column_name = ["dev1","dev2","dev3","dev4","time","locate","cluster"]
time_max = max_value(df_list)

for df in df_list:
    locate = df["locate"].iloc[1]
    x = df["x"].iloc[1]
    y = df["y"].iloc[1]
    mid_df = get_mean(df,column_name,locate,x,y,time_max)
    train_df = pd.concat([train_df,mid_df])

In [11]:
#NANのある行を削除し、time,locate,x,y,clusterをintに変換
direct = train_df[train_df.cluster == 0].dropna(thresh=4).interpolate().dropna()
reflect = train_df[train_df.cluster == 1].dropna(thresh=4).interpolate().dropna()
train = pd.concat([direct,reflect])
train.locate = train.locate.astype("int")
train.time = train.time.astype("int")
train.x = train.x.astype("int")
train.y = train.y.astype("int")
train.cluster = train.cluster.astype("int")
train.count()

dev1       45509
dev2       45509
dev3       45509
dev4       45509
time       45509
locate     45509
x          45509
y          45509
cluster    45509
dtype: int64

### 学習データセットの用意

In [12]:
train_s = test_s = train.sample(frac=1).reset_index(drop=True)
train_s = train_s.iloc[0:len(train)*3//4]
test_s =  test_s.iloc[len(train)*3//4:len(train)]

### 機械学習

In [13]:
from sklearn.utils import shuffle

# 学習用データセットを用意
train_x = np.array(train_s[["dev1","dev2","dev3","dev4","cluster"]])
#train_x = np.array(train_s[["dev2","dev3","dev4","cluster"]])
#train_x = np.array(train_s[["dev1","dev3","cluster"]])
train_y = np.array(train_s[["locate"]])
train_x,train_y = shuffle(train_x,train_y)
test_x = np.array(test_s[["dev1","dev2","dev3","dev4","cluster"]])
#test_x = np.array(test_s[["dev2","dev3","dev4","cluster"]])
#test_x = np.array(test_s[["dev1","dev3","cluster"]])
test_y = np.array(test_s[["locate"]])
test_x,test_y = shuffle(test_x,test_y)

### ランダムフォレスト 

In [17]:
from sklearn.ensemble import RandomForestClassifier
#from sklearn.metrics import (roc_curve, auc, accuracy_score)

model = RandomForestClassifier(n_estimators=100, random_state=0,max_depth=4)

clf = model.fit(train_x,train_y)
pred = clf.predict(test_x)

#fpr, tpr, thresholds = roc_curve(test_y, pred, pos_label=1)
#auc(fpr, tpr)
#accuracy_score(pred, test_y)
hoge = pd.concat([pd.DataFrame(pred),pd.DataFrame(test_y)],axis=1)
hoge.columns = ["pred","answer"]

score = len(hoge[hoge.pred==hoge.answer])/len(hoge)
print("精度:%s" % score)
hoge

/Users/tomanan/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:6: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


精度:0.7510986113552469


,pred,answer
0,4,4
1,6,6
2,6,5
3,8,8
4,1,1
5,7,7
6,1,2
7,5,8
8,1,1
9,1,1


In [18]:
#hoge[hoge.answer == 3].pred.unique()
a = hoge.answer.unique()
a.sort()
for i in a:
    print("\n",i)
    b = hoge[hoge.answer == i].pred.unique()
    b.sort()
    for f in b:
        print(f,hoge[(hoge.answer == i) & (hoge.pred == f)].pred.count())


 1
1 1001
2 58
3 111
4 26
8 2

 2
1 390
2 397
3 325
4 5
5 1
6 3
7 1
8 1

 3
1 18
2 48
3 1038
4 12
5 2
8 23

 4
1 178
2 7
3 37
4 891
5 2
6 1
7 1
8 21

 5
1 1
5 1081
6 4
8 1

 6
1 2
2 5
3 29
4 1
5 97
6 765
7 187
8 31
9 25

 7
3 108
5 28
6 95
7 832
9 30
10 60

 8
1 9
3 208
4 42
5 164
6 39
7 3
8 618
9 53
10 8

 9
3 47
5 42
6 5
7 33
8 1
9 904
10 69

 10
3 2
5 3
6 3
7 47
9 77
10 1019


### XGBoost

In [15]:
#xの予測
import xgboost as xgb
model = xgb.XGBClassifier()
clf = model.fit(train_x,train_y)
pred = clf.predict(test_x)

#fpr, tpr, thresholds = roc_curve(test_y, pred, pos_label=1)
#auc(fpr, tpr)
#accuracy_score(pred, test_y)
hoge = pd.concat([pd.DataFrame(pred),pd.DataFrame(test_y)],axis=1)
hoge.columns = ["pred_locate","answer_locate"]

score = len(hoge[hoge.pred_locate==hoge.answer_locate])/len(hoge)
print("精度:%s" % score)
hoge

ModuleNotFoundError: No module named 'xgboost'

In [50]:
train

,dev1,dev2,dev3,dev4,time,locate,x,y,cluster
10,-90.900000,-73.500000,-76.000000,-81.000000,5,1,200,200,0
12,-90.800000,-70.500000,-77.333333,-77.000000,6,1,200,200,0
14,-90.700000,-71.000000,-78.666667,-73.000000,7,1,200,200,0
16,-90.600000,-73.000000,-80.000000,-79.500000,8,1,200,200,0
18,-90.500000,-70.000000,-75.000000,-81.000000,9,1,200,200,0
20,-90.400000,-74.500000,-75.000000,-82.500000,10,1,200,200,0
22,-90.300000,-73.750000,-75.000000,-76.000000,11,1,200,200,0
24,-90.200000,-73.000000,-83.333333,-81.000000,12,1,200,200,0
26,-90.100000,-72.500000,-76.000000,-80.500000,13,1,200,200,0
28,-90.000000,-72.000000,-81.500000,-80.000000,14,1,200,200,0
